# Bayesian Causal Inference in Non-Randomized Experiments

**Author**: Leo Guelman

* [1. Problem Statment](#problem1)
    * [1.1 The National Study of Learning Mindsets](#mindsets11)
    * [1.2 Data Description](#data12)
    * [1.2 The Questions](#questions13)
* [2. Analysis](#analysis2) 
    * [2.1 Imports](#imports21)
    * [2.2 Data](#data22)
    * [2.3 Assessing Balance of Covariates](#balance23)
    * [2.4 Propensity score](#propscore24)
    * [2.5 Model-based Inference](#mbinference25)
        * [2.5.1 MCMC Diagnostics](#mcmcdiagn251)
        * [2.5.2 Effectivness of Intervention](#effint252)
        * [2.5.3 Treatment effect variation across](#hetero253)
 
 

# 1. Problem Statment <a class="anchor" id="problem1"></a>

## 1.1 The National Study of Learning Mindsets <a class="anchor" id="mindsets11"></a>

We look at the causal inference challenge presented by the *National Study of Learning Mindsets* (Yeager et al., 2019) from a Bayesian perspective. 

The NSLM is a randomized experiment designed to assess the effectiveness of an intervention to improve academic outcomes of students with a *growth mindset*. The *growth mindset* is a belief that people can develop intelligence, as opposed to the *fixed mindset* view which sees intelligence as an innate trait that is fixed at birth.

The original study consisted in a randomized experiment composed of students from 76 schools drawn from the national probability sample of U.S. public schools. In addition, to assessing the average treatment effect (ATE), the study was designed to estimate the degree of heterogeneity in treatment effect across both students and schools. 

A synthetic dataset was generated to mimic the original data, but with the goal of creating an observational study that includes confounding effects not present in the original randomized experiment. Besides this difference, the synthetic data resembles the real NSLM data in terms of covariate distribution, data structures, and effect sizes. 

During the 2018 Atlantic Causal Inference Conference (ACIC 2018), eight groups of participans were invited to analyze the synthetic data to assess the questions of average treatment effect and treatment effect variation in non-randomized experimental settings. Participants employed a diverse set of methods, ranging from matching and flexible outcome modeling to semiparametric estimation and ensemble approaches. In this study, we employ an alternative approach founded in Bayesian inference principles.

## 1.2 Data Description <a class="anchor" id="data12"></a>

The analysis is based on the sythetic dataset of $n=10,391$ children from a sample of $J=76$ schools. For each children $i=\{1, \ldots, n\}$, we observe a binary treatment indicator $Z_i$, a real-valued outcome $Y_i$, as well as 10 categorical or real-valued covariates as outlined in the table below. For a full description of the data generating process refer to Carvalho et al., 2019.


| Covariate | Description |
| :---        |    :----   | 
| S3 | Student’s self-reported expectations for success in the future, a proxy for prior achievement, measured prior to random assignment|
| C1 | Categorical variable for student race/ethnicity |
|C2 | Categorical variable for student identified gender
|C3 | Categorical variable for student first-generation status, i.e. first in family to go to college
|XC | School-level categorical variable for urbanicity of the school, i.e. rural, suburban, etc.
| X1 | School-level mean of students’ fixed mindsets, reported prior to random assignment
| X2|  School achievement level, as measured by test scores and college preparation for the previous 4 cohorts of students
|X3  | School racial/ethnic minority composition, i.e., percentage of student body that is Black, Latino, or Native American
| X4 | School poverty concentration, i.e., percentage of students who are from families whose incomes fall below the federal poverty line
| X5 | School size, i.e., total number of students in all four grade levels in the school
| Y | Post-treatment outcome, a continuous measure of achievement
|Z | Treatment, i.e., receipt of the intervention





## 1.3 The Questions <a class="anchor" id="questions13"></a>

The two questions we are aiming to address as part of this study are the following:

1. Was the mindset intervention effective in improving student achievement?
2. Was the effect of the intervention moderated by school level achievement (`X2`) or pre-existing mindset norms (`X1`)? In particular there are two competing hypotheses about how `X2` moderates the effect of the intervention: Either it is largest in middle-achieving schools (a "Goldilocks effect") or is decreasing in school-level achievement.


# 2. Analysis <a class="anchor" id="analysis2"></a>

## 2.1 Imports <a class="anchor" id="imports21"></a>

In [ ]:
import os
os.chdir('/Users/lguelman/Library/Mobile Documents/com~apple~CloudDocs/LG_Files/Development/BCI/python')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
parameters = {'figure.figsize': (8, 4),
              'font.size': 8, 
              'axes.labelsize': 12}
plt.rcParams.update(parameters)
plt.style.use('fivethirtyeight')
from IPython.display import Image

import pystan
import multiprocessing
import stan_utility
import arviz as az

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

import seaborn as sns

from acic_utils import pre_process_data, stan_model_summary

## 2.2 Data  <a class="anchor" id="data22"></a>

In [ ]:
df = pd.read_csv("../data/synthetic_data.csv")
df
df.info()
df.describe()

## 2.3 Assessing Balance of Covariates <a class="anchor" id="balance23"></a>

Covariate balance is the degree to which the distribution of covariates is similar across levels of the treatment. Here we assess the extent to which the treatment assignment was uniformly randomized across observational units, or there are some selection effects. To that end, we use *Prognostic scores* (Hansen 2008). The prognostic score is defined as the predicted outcome under the control condition, reflecting the baseline "risk", i.e., $E(Y|X, Z=0)$. It is estimated by fitting a model of the outcome in the control group, and then using that model to obtain predictions of the outcome under the control condition for all individuals. The standardize difference in the mean prognostic scores between treatment and control groups is used as a measure of covariate balance. 

Here we fit a Bayesian linear regression model to get a posterior distribution of the standardize difference in the mean prognostic scores between treatment and control groups.

We first pre-process the data (encode categorical features and scaling).

In [ ]:
X, z, y, *_ = pre_process_data(df, standardize_x=False, interactions=False, 
                               p_score=None, drop_first=False)

print("Features dimension:", X.shape)
print("Treatment dimension:", z.shape)
print("Response dimension:", y.shape)
print("Number of treated / control units:", sum(z), "/", X.shape[0]-sum(z))

We now fit the model in [stan](https://mc-stan.org/). We place the stan code separately in `stan_linear_reg.stan`, stored in the repo.

In [ ]:
n, p = X[z==0,:].shape # Fit model using control units only

stan_data_dict = {'N': n,
                  'K': p,
                  'x': X[z==0,:],
                  'y': y[z==0],
                  'N_new': X.shape[0],
                  'x_new': X
                  }

sm = pystan.StanModel('../stan/stan_linear_reg.stan') 
multiprocessing.set_start_method("fork", force=True)
fit = sm.sampling(data=stan_data_dict, iter=1000, chains=4)

In [ ]:
fit_summary = stan_model_summary(fit)
fit_summary

The standardize mean difference in Prognostic scores is positive meaning that students with highest potential outcomes under control are more likely to receive treatment. This can also be appreciated by plotting the proportion of individuals assigned to treatment for by quantile of the prognostic score (each quantile comprises about 1/5 of the observations). In this case, we say that the treatment assignment mechanisms is *counfounded* with the potential outcomes (i.e., the value of the outcome for each subject under each treatment, only one of which is observed).

In this context, a simple comparison of treated versus control individuals would produce bias estimates of treatment effects (both average and conditional effects). We thus proceed the analysis as an observational study instead on a randomized one. Specifically, to address this problem we directly incorporate an estimate of the *propensity score* in the specification of the outcome model, implicitly inducing a covariate dependent prior on the regression function (see Hahn 2020).




In [ ]:
# Extract prognostic scores
samples = fit.extract(permuted=True)
prog_scores = samples['prog_scores'].T

# Compute mean and standardize mean differences in scores
mcmc_samples = prog_scores.shape[1]
prog_scores_diff = np.zeros(mcmc_samples)
prog_scores_std_diff = np.zeros(mcmc_samples)

for s in range(mcmc_samples):
    prog_scores_diff[s] = np.mean(prog_scores[z==1,s]) - np.mean(prog_scores[z==0,s])
    prog_scores_std_diff[s] = prog_scores_diff[s] / np.std(prog_scores[:,s])
  
                               
plt.hist(prog_scores_std_diff, bins = 30)
plt.title("Standardized mean difference in Prognostic scores", fontsize=12)
plt.show()  


In [ ]:
# Compute proportion treated by mean prognostic score quantile
prog_scores_df = pd.DataFrame({'prog_score_mean': np.mean(prog_scores, axis =1),'z':z})
prog_scores_df['prog_score_mean_quantile']= pd.qcut(prog_scores_df['prog_score_mean'], 
                                                    q = 5, labels = False)+1
prog_scores_df.groupby(['prog_score_mean_quantile'])['z'].mean().plot(xticks=list(range(1,6)), 
                                                                     xlabel='Mean Prognostic Score (Quantile)',
                                                                     ylabel='Proportion Treated')

## 2.4 Propensity Score <a class="anchor" id="propscore24"></a>

The propensity score is the probability of treatment assignment conditional on observed baseline covariates, i.e., $E(Z|X)$. Here we fit the propensity score using a flexible non-linear specification based on a Gradient Boosting method. Parameter tuning is done via cross-validated random search.

In [ ]:
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

n_folds = 3
param_n_picks = 30

xgb = XGBClassifier(learning_rate=0.01, n_estimators=500, objective='binary:logistic',
                    silent=True, nthread=1)

skf = StratifiedKFold(n_splits=n_folds, shuffle = True, random_state = 42)

xgb_fits = RandomizedSearchCV(xgb, param_distributions=param_grid,
                              n_iter=param_n_picks, scoring='roc_auc', n_jobs=-1, 
                              cv=skf.split(X,z), verbose=3, random_state=42)

xgb_fits.fit(X, z)

print('\n Best estimator:')
print(xgb_fits.best_estimator_)
print('\n Best AUC score:')
print(xgb_fits.best_score_)
print('\n Best hyperparameters:')
print(xgb_fits.best_params_)

# We now fit the best estimator to all train data 
best_fit = xgb_fits.best_estimator_.fit(X, z)
pscore = best_fit.predict_proba(X)[:,1]
log_odds_pscore = np.log(pscore /(1-pscore))

In [ ]:
prop_score_df = pd.DataFrame({'log_odds_pscore': log_odds_pscore, 'Z':z})
sns.displot(prop_score_df, x="log_odds_pscore", 
            hue="Z",  stat="density", common_norm=False)

## 2.5 Model-based Inference <a class="anchor" id="mbinference25"></a>


We implement the Bayesian inference framework for causal effects introduced by Rubin (1978). In this framework, each observational unit $i=\{1,\ldots, N\}$ is seen as having a potential outcome for each level of treatment. In the binary treatment case with $Z \in [0,1]$, $Y_i(1)$ and $Y_i(0)$ represent the potential outcomes of unit $i$ under $Z=1$ and 
$Z=0$, respectively. Causal inference is considered a missing data problem since both potential outcomes are never jointly observed. Specifically, observed and missing outcomes can be expressed in terms of the potential outcomes as follows:

\begin{align*} 
Y_i^{\text{obs}} &=  Y_i(1)Z_i + Y_i(0)(1-Z_i) \\ 
Y_i^{\text{mis}} &=  Y_i(1)(1-Z_i) + Y_i(0)Z_i
\end{align*}


From a Bayesian perspective, $Y_i^{\text{mis}}$ is considered a latent variable, similar to any other latent variable in the model (I'm trying to avoid the term "parameters" here, which is more common in the frequentist language). The missing potential outcomes can be imputed by estimating their posterior predictive distribution given the observed data. That is,

$$
\text{Pr}(Y^{\text{mis}}| Y^{\text{obs}},Z, X).
$$

Estimating the conditional distribution of $Y^{\text{mis}}$ given $(Y^{\text{obs}},Z, X)$, requires building a model of the joint distribution of potential outcomes $(Y(0), Y(1))$. We assume that the true underlying model for the potential outcomes follows a bivariate Gaussian, 

$$
\begin{pmatrix}
Y_i(0) \\
Y_i(1) 
\end{pmatrix} \bigg\lvert~X_i, \theta \sim N \left(\begin{pmatrix}
\alpha + X_i\beta_c \\
\alpha + X_i\beta_t + \tau
\end{pmatrix},\begin{pmatrix}
\sigma_c^{2} & \rho\sigma_c\sigma_t \\
\rho\sigma_c\sigma_t & \sigma_t^{2}
\end{pmatrix}\right),
$$

where $\theta=(\alpha, \beta_c, \beta_t, \tau, \sigma_c^2, \sigma_t^2, \rho)$.

From this model, we can derive the conditional distribution of each potential outcome as


\begin{align*} 
\text{Pr}(Y_i(1) | Y_i(0), \theta, Z_i=0) &\sim N \Big(\mu_t+ \rho \frac{\sigma_t}{\sigma_c} (Y_i(0)-\mu_c), \sigma^2_t(1-\rho^2)\Big),\\
\text{Pr}(Y_i(0) | Y_i(1), \theta, Z_i=1) &\sim N \Big(\mu_c+ \rho \frac{\sigma_c}{\sigma_t} (Y_i(1)-\mu_t), \sigma^2_c(1-\rho^2)\Big), 
\end{align*}

where $\mu_c = \alpha + X\beta_c$, and $\mu_t = \alpha + X\beta_t + \tau$.

From the distribution of potential outcomes, we can infer the distribution of any estimand of interest of the form $\tau = \tau(Y(0),Y(1), X, Z)$. For instance, the average treatment effect (ATE) can be obtained by simply computing $\frac{1}{N}\sum_{i=1}^N(Y_i(1)-Y_i(0))$. This is also known as the "finite sample" ATE. If instead, we view the observations as a sample from an infinite super-population, then the super-population ATE is given from the posterior distribution of $\tau$.

A few observations from the model above:

* Since $Y_i(0)$ and $Y_i(1)$ are never jointly observed, the correlation between outcomes, $\rho$, cannot be estimated empirically. It must be based on subject-matter knowledge. Sometimes we may choose to be "conservative" about this dependence and therefore assume the worst case. In terms of the posterior variance, the worst case is often the situation of perfect correlation between the two potential outcomes.

* There are two sources of uncertainty in the predictive distribution of the missing potential outcomes: the first is the uncertainty in the estimated latent variables (a.k.a., *epistemic uncertainty*), and the second in the uncertainty in the data as expressed by the Gaussian random sampling mechanism (a.k.a., *aleatoric uncertainty*).

* We allow for heterogeneous treatment effects - i.e., we define two different vectors $\beta_c$ and  $\beta_t$, for control and treated units, respectively. In the **Stan** model below (see `stan_mbi.stan`), this heterogeneity is expressed by including interaction terms between each covariate and the treatment. The posterior estimates on the interaction effects represent the incremental effect of treatment $\beta_t - \beta_c$.

* The propensity score is considered as an additional covariate in $X$.



In [ ]:
X,z, y, a_effects, m_effects, i_effects = pre_process_data(df, standardize_x=False, interactions=True, 
                                                            p_score=pscore, drop_first=False)

# Get indexes of main and interaction effects
idx_m_effects = [a_effects.index(i) for i in m_effects]
idx_i_effects = [a_effects.index(i) for i in i_effects]

print(X.shape)
print(len(idx_m_effects))
print(len(idx_i_effects))
print(X[:,idx_m_effects].shape)
print(X[:,idx_i_effects].shape)

In [ ]:
stan_data_mbi = {'N': X.shape[0], 
                 'N_main_cov':len(idx_m_effects),
                 'N_inter_cov':len(idx_i_effects),
                 'y': y,
                 'z': z,
                 'x': X[:,idx_m_effects],
                 'xz_inter': X[:,idx_i_effects],
                 'rho':0.0}

sm = pystan.StanModel('../stan/stan_mbi.stan') 
multiprocessing.set_start_method("fork", force=True)
fit_mbi = sm.sampling(data=stan_data_mbi, iter=1000, chains=4, seed=194838)

### 2.5.1 MCMC Diagnostics <a class="anchor" id="mcmcdiagn251"></a>


In [ ]:
summary_stan_fit = stan_model_summary(fit_mbi)
summary_stan_fit

In [ ]:
r_hat = summary_stan_fit['Rhat']
r_hat.plot.hist(title="Rhat")
plt.axvline(1.1, color='r', linestyle='--')

In [ ]:
summary_stan_fit['n_eff'].plot.hist(title="n_eff")

In [ ]:
stan_utility.utils.check_treedepth(fit_mbi)

In [ ]:
stan_utility.utils.check_energy(fit_mbi)

In [ ]:
stan_utility.utils.check_div(fit_mbi)

### 2.5.2  Effectiveness of Intervention<a class="anchor" id="effint252"></a>

Here we address the first study question, which we recall is "*Was the mindset intervention effective in improving student achievement?*" This relates to the *Average Treatment Effect (ATE)*. The estimated (finite-sample) **ATE is 0.24 with a 95% uncertainty interval between 0.22 and 0.26. The true value based on the simulation is also 0.24**, so we are right on spot. 

In [ ]:
taus = summary_stan_fit.loc[['tau_fs']]
taus

For comparison purposes, we show below the submitted estimates for average treatment effects and corresponding 95% uncertainty intervals from eight ACIC 2018 challenge participants (Carvalho 2019). Our estimate is closer to the true value relative to all submissions.

In [ ]:
Image(filename = "../img/acic_ate.png", width = 300, height = 200)

### 2.5.3  Treatment effect variation across `X1` and `X2` <a class="anchor" id="hetero253"></a>

The second question of the study is directed to assess the treatment effect variation across the two pre-specified moderators, `X1` (pre-existing mindset norms) and `X2` (school level achievement).

We look at the posterior estimates of interaction effects `Z x X1` and `Z x X2`. This effect represent the incremental effect of treatment $\beta_t - \beta_c$ from `X1` and `X2`, respectively. The results depicted below indicate that a higher baseline mindset beliefs tends to be associated with lower treatment effect on average. In contrast, there appears to be no support for treatment effect variation across school achievement levels. 

In [ ]:
samples = fit_mbi.extract(permuted=True)

Z_X1_samples = samples['beta_inter'][:,i_effects.index('Z_X1')]
Z_X2_samples = samples['beta_inter'][:,i_effects.index('Z_X2')]

g1 = sns.displot(Z_X1_samples,
    kind="kde")
g1.set_axis_labels("Interaction effect (Z x X1) posterior samples", 
                  "Density", fontsize=10)

g2 = sns.displot(Z_X2_samples,
    kind="kde")
g2.set_axis_labels("Interaction effect (Z x X2) posterior samples", 
                  "Density", fontsize=10)

# References <a class="anchor" id="ref"></a>

Carvalho, C., Feller, A., Murray, J., Woody, S., and Yeager, D. Assessing Treatment Effect Variation in Observational Studies: Results from a Data Challenge, (2019). https://arxiv.org/abs/1907.07592

Donald B. Rubin. Bayesian Inference for Causal Effects: The Role of Randomization. Ann. Statist. 6 (1) 34 - 58 (1978). https://doi.org/10.1214/aos/1176344064

Hahn, P.R., Murray, J.S., Carvalho, C.M. Bayesian Regression Tree Models for Causal Inference: Regularization, Confounding, and Heterogeneous Effects (with Discussion). Bayesian Analysis. 15 (3) 965 - 1056 (2020). https://doi.org/10.1214/19-BA1195

Hansen, Ben B. The Prognostic Analogue of the Propensity Score. Biometrika 95 (2), 481–88, (2008). https://doi.org/10.1093/biomet/asn004.

Yeager, D.S., Hanselman, P., Walton, G.M. et al. A national experiment reveals where a growth mindset improves achievement. Nature 573, 364–369 (2019). https://doi.org/10.1038/s41586-019-1466-y